In [2]:
bucket_name = "sk2224-projectdata"
!aws s3 mb s3://{bucket_name}

make_bucket: sk2224-projectdata


In [3]:
# Setup - Run only once per Kernel App
%conda install openjdk -y

# install PySpark
%pip install pyspark==3.4.0

# install spark-nlp
%pip install spark-nlp==5.1.3

# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

Retrieving notices: ...working... done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.10.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.10.0



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - openjdk


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2023.08.22 |       h06a4308_0         123 KB
    certifi-2023.11.17         |  py310h06a4308_0         158 KB
    openjdk-11.0.13            |       h87a67e3_0       341.0 MB
    ------------------------------------------------------------
                                           Total:       341.3 MB

The following NEW packages will be INSTALLED:

  openjdk            pkgs/main/

In [2]:
# Import pyspark and build Spark session
from pyspark.sql import SparkSession

# Import pyspark and build Spark session
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","16G")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.3,org.apache.hadoop:hadoop-aws:3.2.2")\
    .config(
            "fs.s3a.aws.credentials.provider",
            "com.amazonaws.auth.ContainerCredentialsProvider",
    )\
    .getOrCreate()

:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-aaf659d4-1472-4258-b281-9ab7abc347b7;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.1.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.20.1 in central
	found com.google.guava#guava;31.1-jre in c

In [4]:
import sagemaker
from pyspark.sql.functions import lower, regexp_replace, col, concat_ws
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.base import Finisher, DocumentAssembler

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [5]:
%%time
bucket = "sk2224-projectdata"
session = sagemaker.Session()
output_prefix_data_comments = "yyyy=*"
s3_path = f"s3a://{bucket}/{output_prefix_data_comments}"
print(f"reading submissions from {s3_path}")
submissions = spark.read.parquet(s3_path, header=True)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
reading submissions from s3a://sk2224-projectdata/yyyy=*


23/11/20 21:48:58 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


CPU times: user 265 ms, sys: 7.58 ms, total: 272 ms
Wall time: 12.8 s


23/11/20 21:49:10 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
submissions.printSchema()

root
 |-- adserver_click_url: string (nullable = true)
 |-- adserver_imp_pixel: string (nullable = true)
 |-- archived: boolean (nullable = true)
 |-- author: string (nullable = true)
 |-- author_cakeday: boolean (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- brand_safe: boolean (nullable = true)
 |-- contest_mode: boolean (nullable = true)
 |-- created_utc: timestamp (nullable = true)
 |-- crosspost_parent: string (nullable = true)
 |-- crosspost_parent_list: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- approved_at_utc: string (nullable = true)
 |    |    |-- approved_by: string (nullable = true)
 |    |    |-- archived: boolean (nullable = true)
 |    |    |-- author: string (nullable = true)
 |    |    |-- author_flair_css_class: string (nullable = true)
 |    |    |-- author_flair_text: string (nullable = true)
 |    |    

In [6]:
# Filter out rows where 'text' or 'author' is '[deleted]'
submissions_filtered = submissions.filter((submissions.selftext != '[deleted]') & (submissions.selftext != '[removed]') & (submissions.author != '[deleted]') & (submissions.author != '[removed]')& (submissions.title != '[deleted]') &  (submissions.title != '[removed]')) 

# Show the filtered DataFrame
submissions_filtered = submissions_filtered.select("subreddit", "author", "title", "selftext",
                             "created_utc", "num_comments", "score", 
                             "over_18", "media", "pinned", "locked", 
                             "disable_comments", "domain", "hidden", 
                             "distinguished", "hide_score")

In [18]:
submissions_filtered.show(5)

+----------+----------+--------------------+--------------------+-------------------+------------+-----+-------+-----+------+------+----------------+---------------+------+-------------+----------+
| subreddit|    author|               title|            selftext|        created_utc|num_comments|score|over_18|media|pinned|locked|disable_comments|         domain|hidden|distinguished|hide_score|
+----------+----------+--------------------+--------------------+-------------------+------------+-----+-------+-----+------+------+----------------+---------------+------+-------------+----------+
|television| Hyhyhyhuh|Is there a websit...|Hey all,  \n\n\nI...|2021-01-27 22:16:47|           9|    0|  false| null| false| false|            null|self.television| false|         null|     false|
|television|boxermansr|‘Doogie Kameāloha...|                    |2021-01-27 22:17:42|          11|    4|  false| null| false| false|            null|   deadline.com| false|         null|     false|
|     anim

In [7]:
from pyspark.sql.functions import concat_ws

# Combine 'title' and 'selftext' columns into a new column 'RedditText'
submissions_combined = submissions_filtered.withColumn(
    "RedditText", concat_ws(" ", "title", "selftext")
)

In [8]:
# Show the first 5 rows of the DataFrame with the new column
submissions_combined.select("subreddit", "author", "RedditText", "created_utc", "num_comments", "score").show(5)

+----------+----------+--------------------+-------------------+------------+-----+
| subreddit|    author|          RedditText|        created_utc|num_comments|score|
+----------+----------+--------------------+-------------------+------------+-----+
|television| Hyhyhyhuh|Is there a websit...|2021-01-27 22:16:47|           9|    0|
|television|boxermansr|‘Doogie Kameāloha...|2021-01-27 22:17:42|          11|    4|
|     anime|    Mjrbks|Ever suddenly fin...|2021-01-27 22:20:14|           7|    0|
|    movies|Swampcrone|Cloris Leachman, ...|2021-01-27 22:20:20|           0|    1|
|television|jaykay1107|Netflix, Alan Yan...|2021-01-27 22:20:56|           0|    2|
+----------+----------+--------------------+-------------------+------------+-----+
only showing top 5 rows



In [9]:
submissions_combined = submissions_combined.select("subreddit", "author", "RedditText", "created_utc", "num_comments", "score")

In [10]:
from sparknlp.annotator import (Tokenizer, Normalizer,
                                LemmatizerModel, StopWordsCleaner)

In [11]:
documentAssembler = DocumentAssembler()\
    .setInputCol("RedditText")\
    .setOutputCol("document")

In [12]:
# Regex Tokenizer to break words
tokenizer = Tokenizer() \
     .setInputCols(['document']) \
     .setOutputCol('token')

In [13]:
# Normalizing and setting case insensitive to be true
normalizer = Normalizer() \
     .setInputCols(['token']) \
     .setOutputCol('normalized') \
     .setLowercase(True)

In [10]:
!pip install nltk


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [14]:
import nltk
from nltk.corpus import stopwords

# Download stopwords data
nltk.download('stopwords')

# Now you can use stopwords from nltk.corpus
stopwords_list = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
# Remove Stopwords
stopwords_cleaner = StopWordsCleaner() \
     .setInputCols(['lemma']) \
     .setOutputCol('clean_lemma') \
     .setCaseSensitive(False) \
     .setStopWords(stopwords_list)

In [16]:
# finisher converts tokens to human-readable output
finisher = Finisher() \
     .setInputCols(['clean_lemma']) \
     .setCleanAnnotations(False)

In [17]:
# Lemmatizing
lemmatizer = LemmatizerModel.pretrained() \
     .setInputCols(['normalized']) \
     .setOutputCol('lemma')

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[ / ]lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
Download done! Loading the resource.
[ — ]

[OK!]


In [18]:
pipeline = Pipeline() \
    .setStages([
        documentAssembler,
        tokenizer,
        normalizer,
        lemmatizer,  # Add the lemmatizer stage here
        stopwords_cleaner.setInputCols(['lemma']),
        finisher
    ])


In [19]:
from pyspark.ml import PipelineModel

# Create an empty DataFrame with the same schema as your actual data
empty_df = spark.createDataFrame([], schema=submissions_combined.schema)

In [20]:
# Fit the pipeline model on the empty DataFrame
pipeline_model = pipeline.fit(empty_df)

In [21]:
# Transform your actual DataFrame using the fitted pipeline model
submission_clean = pipeline_model.transform(submissions_combined)

In [57]:
submission_clean.select(submission_clean.finished_clean_lemma).show(5)

+--------------------+
|finished_clean_lemma|
+--------------------+
|[website, allow, ...|
|[doogie, kameāloh...|
|[ever, suddenly, ...|
|[cloris, leachman...|
|[netflix, alan, y...|
+--------------------+
only showing top 5 rows



In [23]:
submission_clean.columns

['subreddit',
 'author',
 'RedditText',
 'created_utc',
 'num_comments',
 'score',
 'clean_RedditText']

In [22]:
# Remove 'lemma', 'clean_lemma' from the DataFrame
submission_clean = submission_clean.drop('lemma', 'clean_lemma',
 'document',
 'token',
 'normalized',)

# Rename 'finished_clean_lemma' to 'clean_RedditText'
submission_clean = submission_clean.withColumnRenamed('finished_clean_lemma', 'clean_RedditText')


In [71]:
submission_clean.show(5)

+----------+----------+--------------------+-------------------+------------+-----+--------------------+
| subreddit|    author|          RedditText|        created_utc|num_comments|score|    clean_RedditText|
+----------+----------+--------------------+-------------------+------------+-----+--------------------+
|television| Hyhyhyhuh|Is there a websit...|2021-01-27 22:16:47|           9|    0|[website, allow, ...|
|television|boxermansr|‘Doogie Kameāloha...|2021-01-27 22:17:42|          11|    4|[doogie, kameāloh...|
|     anime|    Mjrbks|Ever suddenly fin...|2021-01-27 22:20:14|           7|    0|[ever, suddenly, ...|
|    movies|Swampcrone|Cloris Leachman, ...|2021-01-27 22:20:20|           0|    1|[cloris, leachman...|
|television|jaykay1107|Netflix, Alan Yan...|2021-01-27 22:20:56|           0|    2|[netflix, alan, y...|
+----------+----------+--------------------+-------------------+------------+-----+--------------------+
only showing top 5 rows

